In [1]:
import numpy as np
import pandas as pd
import sqlite3  # Import sqlite3 since we are working with SQL


In [2]:
# Import the given dataset to adult_df
adult_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)

In [3]:
#Create the column names as following as in https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
adult_df.columns = ['age', 'workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','annual-salary']
adult_df.head(2)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual-salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


Create a sql db from adult dataset and name it sqladb

In [9]:
conn = sqlite3.connect('sqladb')   # Create a connection using sqlite3 connect command to 'sqladb' database
cursor = conn.cursor()             # Create a cursor variable which acts as a cursor for sqlite
adult_df.to_sql('adult_data',conn) # Create a adult_data table in sqladb database using the contents of adult_df

Select 10 records from the adult sqladb

In [10]:
query = """SELECT * FROM adult_data LIMIT 10;"""
pd.read_sql(query, conn)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual-salary
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


Show me the average hours per week of all men who are working in private sector

In [11]:
query = """SELECT avg("hours-per-week") as Avg_Hours FROM adult_data where sex = 'Male' and workclass = 'Private'"""
pd.read_sql(query, conn)

,Avg_Hours
0,42.221226


Show me the frequency table for education, occupation and relationship, separately

In [12]:
query = """SELECT education , COUNT(education) as edu_frequency FROM adult_data GROUP BY education"""
pd.read_sql(query,conn)

,education,edu_frequency
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5355


In [13]:
query = """SELECT occupation , COUNT(occupation) as ocu_frequency FROM adult_data GROUP BY occupation"""
pd.read_sql(query,conn)

,occupation,ocu_frequency
0,?,1843
1,Adm-clerical,3770
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [14]:
query = """SELECT relationship , COUNT(relationship) as rel_frequency FROM adult_data GROUP BY relationship"""
pd.read_sql(query,conn)

,relationship,rel_frequency
0,Husband,13193
1,Not-in-family,8305
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


Are there any people who are married, working in private sector and having a masters
degree

In [16]:
query = """SELECT count(*) as Count from adult_data where ("marital-status" <>'Never-married' or "marital-status" <> 'Divorced') 
            and workclass='Private' and education='Masters'"""
pd.read_sql(query,conn)

,Count
0,894


What is the average, minimum and maximum age group for people working in
different sectors

In [19]:
query = """SELECT workclass,avg(age) Avg_age,min(age) MinAge,max(age) MaxAge from adult_data GROUP BY workclass"""
pd.read_sql(query,conn)

,workclass,Avg_age,MinAge,MaxAge
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


Calculate age distribution by country

In [20]:
query = """SELECT "native-country",avg(age) Avg_age,min(age) MinAge,max(age) MaxAge from adult_data GROUP BY "native-country" """
pd.read_sql(query,conn)

,native-country,Avg_age,MinAge,MaxAge
0,?,38.725557,17,90
1,Cambodia,37.789474,18,65
2,Canada,42.545455,17,80
3,China,42.533333,22,75
4,Columbia,39.711864,18,75
5,Cuba,45.768421,21,82
6,Dominican-Republic,37.728571,18,78
7,Ecuador,36.642857,21,90
8,El-Salvador,34.132075,17,79
9,England,41.155556,17,90


Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
'capital-loss'

In [21]:
query = """SELECT "capital-gain", "capital-loss", "capital-gain" + "capital-loss" as 'Net-Capital-Gain' from adult_data """
pd.read_sql(query,conn)

,capital-gain,capital-loss,Net-Capital-Gain
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,14084,0,14084
9,5178,0,5178
